<a href="https://colab.research.google.com/github/tomonari-masada/course-nlp2020/blob/master/11_topic_modeling(gensim).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# gensimでLDAを動かす

* gensimのLdaModelはデフォルトの設定だと正しく動かない
 * passesを20ぐらいにはしておくこと。
 * 下記Webページは使い方を間違っているので要注意（passesをデフォルト設定で使っている）
 http://www.ie110704.net/2018/12/29/wordcloud%E3%81%A8pyldavis%E3%81%AB%E3%82%88%E3%82%8Blda%E3%81%AE%E5%8F%AF%E8%A6%96%E5%8C%96%E3%81%AB%E3%81%A4%E3%81%84%E3%81%A6/
* gensimのperplexityはトークンあたりのELBOのnp.exp2()で求めている
 * 自然対数の底を使って求めたELBOをもとにして計算しているにもかかわらず。


In [ ]:
import os.path
import re
import tarfile
import smart_open


PATH = '/content/drive/MyDrive/data'


def extract_documents(url='https://cs.nyu.edu/~roweis/data/nips12raw_str602.tgz'):

  fname = os.path.join(PATH, url.split('/')[-1])

  if not os.path.isfile(fname):
    with smart_open.open(url, "rb") as fin:
      with smart_open.open(fname, 'wb') as fout:
        while True:
          buf = fin.read(io.DEFAULT_BUFFER_SIZE)
          if not buf:
            break
          fout.write(buf)

  with tarfile.open(fname, mode='r:gz') as tar:
  # Ignore directory entries, as well as files like README, etc.
    files = [
             m for m in tar.getmembers()
             if m.isfile() and re.search(r'nipstxt/nips\d+/\d+\.txt', m.name)
             ]
    for member in sorted(files, key=lambda x: x.name):
      member_bytes = tar.extractfile(member).read()
      yield member_bytes.decode('utf-8', errors='replace')

In [ ]:
docs = list(extract_documents())

In [ ]:
print(len(docs))

In [ ]:
print(docs[0][:1000])

In [ ]:
import spacy

nlp = spacy.load('en', disable=["tagger", "parser", "ner"])

In [ ]:
def spacy_lemmatize_text(nlp, text):
  text = nlp(text.lower())
  doc = [word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text]
  return [word for word in doc if len(word) > 1] # 長さ1の単語は削除

In [ ]:
from tqdm import tqdm

new_docs = list()
for doc in tqdm(docs):
  new_docs.append(spacy_lemmatize_text(nlp, doc))

In [ ]:
print(new_docs[0])

In [ ]:
from gensim import corpora

In [ ]:
dictionary = corpora.Dictionary(new_docs)

In [ ]:
print(dictionary)

In [ ]:
dictionary.filter_extremes(no_below=10, no_above=0.5)

In [ ]:
len(dictionary)

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in new_docs]

In [ ]:
print(corpus[0])

In [ ]:
n_components = 20

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
from gensim.models.ldamodel import LdaModel

In [ ]:
lda = LdaModel(corpus=corpus, num_topics=n_components,
               passes=20)

In [ ]:
import numpy as np
np.exp(- lda.log_perplexity(corpus))